# Word Embeddings : le modèle Word2Vec

## Imports

In [1]:
import sys

from gensim.models.phrases import Phrases, Phraser
from gensim.models import Word2Vec

import nltk
from nltk.tokenize import wordpunct_tokenize
from unidecode import unidecode

## Chargement et traitement des phrases du corpus

### Création d'un objet qui *streame* les lignes d'un fichier pour économiser de la RAM

In [2]:
class MySentences(object):
    """Tokenize and Lemmatize sentences"""
    def __init__(self, filename):
        self.filename = filename

    def __iter__(self):
        for line in open(self.filename, encoding='utf-8', errors="backslashreplace"):
            yield [unidecode(w.lower()) for w in wordpunct_tokenize(line)]

In [3]:
infile = f"../data/sents_2.txt"
sentences = MySentences(infile)

### Détection des bigrams

Article intéressant sur le sujet : https://towardsdatascience.com/word2vec-for-phrases-learning-embeddings-for-more-than-one-word-727b6cf723cf

In [4]:
bigram_phrases = Phrases(sentences)

L'object `phrases` peut être vu comme un large dictionnaire d'expressions multi-mots associées à un score, le *PMI-like scoring*. Ce dictionnaire est construit par un apprentissage sur base d'exemples.
Voir les références ci-dessous :
- https://arxiv.org/abs/1310.4546
- https://en.wikipedia.org/wiki/Pointwise_mutual_information

In [5]:
type(bigram_phrases.vocab)

dict

Il contient de nombreuses clés qui sont autant de termes observés dans le corpus

In [6]:
len(bigram_phrases.vocab.keys())

5978799

Prenons une clé au hasard :

In [7]:
key_ = list(bigram_phrases.vocab.keys())[144]
print(key_)

absence_&


Le dictionnaire indique le score de cette coocurrence :

In [8]:
bigram_phrases.vocab[key_]

1

Lorsque l'instance de `Phrases` a été entraînée, elle peut concaténer les bigrams dans les phrases lorsque c'est pertinent.

In [9]:
%time bigram_phrases[sentences]

CPU times: user 1.33 ms, sys: 6 µs, total: 1.34 ms
Wall time: 5.16 ms


### Conversion des `Phrases` en objet `Phraser`

`Phraser` est un alias pour `gensim.models.phrases.FrozenPhrases`, voir ici https://radimrehurek.com/gensim/models/phrases.html.

Le `Phraser` est une version *light* du `Phrases`, plus optimale pour transformer les phrases en concaténant les bigrams.

In [10]:
bigram_phraser = Phraser(phrases_model=bigram_phrases)

Le `Phraser` est un objet qui convertit certains unigrams d'une liste en bigrams lorsqu'ils ont été identifiés comme pertinents.

In [11]:
%time bigram_phraser[sentences]

CPU times: user 3.56 ms, sys: 15 µs, total: 3.57 ms
Wall time: 3.49 ms


### Extraction des trigrams

Nous répétons l'opération en envoyant cette fois la liste de bigrams afin d'extraire les trigrams.

In [12]:
trigram_phrases = Phrases(bigram_phraser[sentences])

In [13]:
trigram_phraser = Phraser(phrases_model=trigram_phrases)

### Création d'un corpus d'unigrams, bigrams, trigrams

In [14]:
corpus = list(trigram_phraser[bigram_phraser[sentences]])

In [15]:
print(corpus[:100])

[['v', 'i', 'l', 'l', 'e', 'de', 'bruxelles', 'bulletin', 'ires', '8eanas', 'dl', '!'], ['conseil_communal', 'annee', '1847', '.'], ['au', 'ville', 'de', 'b', 'r', 'u', 'x', 'e', 'l', 'l', 'e', 's', '.'], ['bulletin', 'conseil', 'aes', 'seances', 'communal', '.'], ['annee', '1847', '.'], ['bruxelles', ',', 'imprimerie', 'd', 'e', 'j', '.'], ['h', '.', 'b', 'r', 'i', 'a', 'r', 'd', ',', 'rite', 'n', 'e', 'u', 'v', 'e', ',', '3', '1', ',', 'faubourg', 'de', 'n', 'a', 'm', 'u', 'r', ',', '1', '84', '8', 'de', '!'], ['du', 'consei', 'dibi', 'e', '.', '-', 'communication', 'conclusions', 'de', 'la', 'section', 'des', 'du', 'nouvel_hospice', 'pour', 'les', 'av', 'enraisonde', 'l', "'", 'absence', '&', 'maladie', '.', 'le', 'conseil', 'ajourne', 'leurs', 'de', 'pierre', 'el', 'marchai', 'cles', 'des', 'taxes', 'communale', "'", 'bieniaance', 'eldeseianv', 'il', 'est', 'donne', 'communie', ';', 'mandant', 'le', 'o', 'p', 'fa', 'gnant', 'l', "'", 'envoi', 'de', 'leur', 'bn', 'par', 'l', "'", 'e

## Entrainement d'un modèle Word2Vec sur ce corpus (modèle 2)

In [16]:
%%time
model = Word2Vec(
    corpus, # On passe le corpus de ngrams que nous venons de créer
    vector_size=32, # Le nombre de dimensions dans lesquelles le contexte des mots devra être réduit, aka. vector_size
    window=5, # La taille du "contexte", ici 5 mots avant et après le mot observé
    min_count=5, # On ignore les mots qui n'apparaissent pas au moins 5 fois dans le corpus
    workers=4, # Permet de paralléliser l'entraînement du modèle en 4 threads
    epochs=5 # Nombre d'itérations du réseau de neurones sur le jeu de données pour ajuster les paramètres avec la descende de gradient, aka. epochs.
)

CPU times: user 19min 5s, sys: 47.3 s, total: 19min 52s
Wall time: 7min 21s


#### Remarque

Vous voyez ici que l'entrainement du modèle est parallélisé (sur 4 workers).

Lors qu'on parallélise l'entrainement du modèle, 4 modèles "séparés" sont entrainés sur environ un quart des phrases.

Ensuite, les résultats sont agrégés pour ne plus faire qu'un seul modèle.

On ne peut prédire quel worker aura quelle phrase, car il y a des aléas lors de la parallélisation (p. ex. un worker qui serait plus lent, etc.).

Du coup, les valeurs peuvent varier légèrement d'un entrainement à l'autre.

Mais, globalement, les résultats restent cohérents.

### Sauver le modèle dans un fichier

In [17]:
outfile = f"../data/bulletins_2.model"
model.save(outfile)

## Explorer le modèle

### Charger le modèle en mémoire

In [28]:
model = Word2Vec.load("../data/bulletins_2.model")

### Imprimer le vecteur d'un terme

In [29]:
model.wv["bruxelles"]

array([ 0.58973897, -1.0262196 ,  2.0716813 ,  1.589865  ,  1.3657871 ,
       -2.0293982 , -0.21928205,  4.652348  , -4.202351  , -4.3337784 ,
       -0.86379164,  5.786408  , -2.7091334 ,  1.221566  ,  3.5432715 ,
        0.9311423 , -1.8224436 ,  1.488655  ,  2.5310202 , -0.78291976,
        0.30947435, -3.211929  , -1.6163418 ,  4.165647  , -3.4496503 ,
        0.3898385 ,  0.67458636,  1.77817   ,  1.7902862 ,  2.8616135 ,
       -3.8638105 , -0.1167161 ], dtype=float32)

### Calculer la similarité entre deux termes

In [30]:
model.wv.similarity("boucher", "boulanger")

0.86908996

### Chercher les mots les plus proches d'un terme donné

In [31]:
model.wv.most_similar("bruxelles", topn=10)

[('laeken', 0.8239486217498779),
 ('considere_comme_officiel', 0.7925914525985718),
 ('liege', 0.7702558040618896),
 ('paris', 0.7679300308227539),
 ('nivelles', 0.7521665096282959),
 ('gand', 0.7397889494895935),
 ('lille', 0.7102484703063965),
 ('differents_quartiers', 0.6752375960350037),
 ('lille_1860', 0.6710933446884155),
 ('louvain', 0.6700588464736938)]

### Faire des recherches complexes à travers l'espace vectoriel

In [32]:
print(model.wv.most_similar(positive=['bruxelles', 'france'], negative=['belgique']))

[('lille', 0.8239251375198364), ('ostende', 0.8021267056465149), ('gand', 0.7774108052253723), ('manchester', 0.7756398320198059), ('londres', 0.7693049311637878), ('tournai', 0.7622190117835999), ('verviers', 0.7603538036346436), ('paris', 0.7556376457214355), ('louvain', 0.7518820762634277), ('prague', 0.7503906488418579)]


### Modèle 3: Taille du vecteur = 100 (vs 32)

In [33]:
%%time
model = Word2Vec(
    corpus, # On passe le corpus de ngrams que nous venons de créer
    vector_size=100, # Le nombre de dimensions dans lesquelles le contexte des mots devra être réduit, aka. vector_size
    window=5, # La taille du "contexte", ici 5 mots avant et après le mot observé
    min_count=5, # On ignore les mots qui n'apparaissent pas au moins 5 fois dans le corpus
    workers=4, # Permet de paralléliser l'entraînement du modèle en 4 threads
    epochs=5 # Nombre d'itérations du réseau de neurones sur le jeu de données pour ajuster les paramètres avec la descende de gradient, aka. epochs.
)

CPU times: user 26min 12s, sys: 1min 6s, total: 27min 19s
Wall time: 9min 32s


In [34]:
outfile = f"../data/bulletins_3.model"
model.save(outfile)

In [35]:
model = Word2Vec.load("../data/bulletins_3.model")

In [36]:
model.wv["bruxelles"]

array([-2.821787  ,  1.7417598 , -0.47517857, -0.9297561 ,  0.20933826,
       -0.33410355,  1.0013304 , -3.9408467 ,  2.2906547 , -1.2519214 ,
        2.0284204 ,  0.4734065 ,  0.7797554 , -2.3764455 ,  2.3031328 ,
        1.3654215 , -0.23531204,  4.263294  ,  2.5052593 ,  1.7040656 ,
        0.7755428 ,  1.1707009 , -3.485699  , -1.4181716 ,  1.7519428 ,
       -1.1885886 , -0.9323578 ,  2.7376783 , -0.05357512,  3.0506134 ,
       -0.04246143,  3.1094618 ,  2.361703  ,  0.3488069 ,  0.13668808,
        1.7061647 ,  1.0303104 ,  0.16420278, -2.3176546 , -1.2321485 ,
        1.0473685 , -0.16010597,  0.21304561, -3.2333903 ,  1.7420102 ,
       -1.5983607 , -1.4976974 ,  0.67025024,  2.446454  ,  2.8469923 ,
       -0.8682723 ,  0.5673862 , -1.9958344 ,  0.12715669, -2.2643695 ,
       -0.83515686,  0.38899907, -2.7805963 ,  1.0056809 ,  2.4225903 ,
       -0.6718316 ,  0.7441178 , -0.16049749,  1.4168892 ,  2.6510496 ,
        0.04205501, -4.032287  ,  4.0189724 , -2.0547545 , -2.65

In [37]:
model.wv.similarity("boucher", "boulanger")

0.7806577

In [38]:
model.wv.most_similar("bruxelles", topn=10)

[('rruxelles', 0.6470252275466919),
 ('paris', 0.6332045793533325),
 ('liege', 0.631273090839386),
 ('nivelles', 0.6280113458633423),
 ('gand', 0.6024729013442993),
 ('laeken', 0.5872405767440796),
 ('lille', 0.5851026773452759),
 ('bruxelle', 0.5699034929275513),
 ('jumeaux_naissances', 0.5477592945098877),
 ('malines', 0.5462380647659302)]

In [39]:
print(model.wv.most_similar(positive=['bruxelles', 'france'], negative=['belgique']))

[('lille', 0.6613479256629944), ('paris', 0.6434320211410522), ('verviers', 0.6389840841293335), ('londres', 0.6370319724082947), ('hambourg', 0.6282646059989929), ('prague', 0.6222774386405945), ('gand', 0.6142219305038452), ('tournai', 0.6134740114212036), ('liege', 0.6133009791374207), ('ostende', 0.6108174920082092)]


### Modèle 4: Vecteur 100 + élargissement de la fenêtre à 20 (vs 5)

In [40]:
%%time
model = Word2Vec(
    corpus, # On passe le corpus de ngrams que nous venons de créer
    vector_size=100, # Le nombre de dimensions dans lesquelles le contexte des mots devra être réduit, aka. vector_size
    window=20, # La taille du "contexte", ici 5 mots avant et après le mot observé
    min_count=5, # On ignore les mots qui n'apparaissent pas au moins 5 fois dans le corpus
    workers=4, # Permet de paralléliser l'entraînement du modèle en 4 threads
    epochs=5 # Nombre d'itérations du réseau de neurones sur le jeu de données pour ajuster les paramètres avec la descende de gradient, aka. epochs.
)

CPU times: user 31min 2s, sys: 1min 10s, total: 32min 12s
Wall time: 10min 3s


In [41]:
outfile = f"../data/bulletins_4.model"
model.save(outfile)

In [117]:
model = Word2Vec.load("../data/bulletins_4.model")

In [118]:
model.wv["bruxelles"]

array([-3.5294814 ,  0.46205315, -3.136256  ,  2.7181191 , -0.19754034,
        1.3294256 , -5.4065156 , -1.7076005 ,  1.1482389 , -1.7121512 ,
        3.2751715 , -6.316165  , -2.3904014 ,  2.6055224 , -4.316526  ,
       -1.3751478 ,  3.7133205 , -0.11183983,  2.466617  , -2.2758758 ,
        6.209551  , -5.186154  ,  2.6362963 ,  4.2062054 ,  1.7750355 ,
       -2.3785162 , -2.730668  , -0.01242101, -4.154819  , -0.3899968 ,
       -0.77142626,  0.82222265, -3.6569283 , -1.4491752 ,  4.992444  ,
       -0.6587832 ,  1.5070369 , -0.5447458 ,  2.6052966 ,  0.9000388 ,
       -3.480406  ,  0.1302352 ,  2.097107  , -3.139999  ,  0.8931701 ,
       -4.249983  , -0.11780771,  1.2884921 ,  3.125195  , -3.7342858 ,
       -6.842377  , -0.7513382 ,  0.7284325 ,  0.5234136 , -1.0563607 ,
       -7.118118  ,  5.3821054 , -4.446206  ,  3.4555452 , -1.8658125 ,
        2.457817  ,  1.0376217 ,  2.5419545 , -3.8404133 ,  2.0115778 ,
       -1.5690126 , -0.6278956 ,  1.6097066 ,  2.5003555 , -0.84

In [119]:
model.wv.similarity("boucher", "boulanger")

0.69779754

In [120]:
model.wv.most_similar("bruxelles", topn=10)

[('xelles', 0.6508788466453552),
 ('dette_active', 0.6217373609542847),
 ('bruxelle', 0.5955502390861511),
 ('liege', 0.586171567440033),
 ('laeken', 0.5658113956451416),
 ('nivelles', 0.559372067451477),
 ('hutel', 0.5444490909576416),
 ('vhotel', 0.5254800915718079),
 ('manufacturiere', 0.5184491276741028),
 ('bruxel', 0.5167596340179443)]

In [121]:
print(model.wv.most_similar(positive=['bruxelles', 'france'], negative=['belgique']))

[('liege', 0.6075229048728943), ('manchester', 0.6038661599159241), ('hambourg', 0.5692122578620911), ('ostende', 0.5674587488174438), ('paris', 0.5573484897613525), ('lille', 0.5570899844169617), ('cologne', 0.5560243129730225), ('gand', 0.5556208491325378), ('amsterdam', 0.5449244976043701), ('tournai', 0.5420806407928467)]


In [123]:
print(model.wv.most_similar(positive=['homme', 'roi'], negative=['femme']))

[('homme_politique', 0.6000778079032898), ('souverain', 0.5965874195098877), ('episode', 0.5854206085205078), ('defenseur', 0.5852083563804626), ('peuple', 0.5814440250396729), ('prince', 0.5742866396903992), ('parlement', 0.5693864822387695), ('magistrat', 0.5646989345550537), ('congres', 0.564418375492096), ('ecria', 0.562084972858429)]


In [124]:
print(model.wv.most_similar(positive=['france', 'belgique'], negative=['paris']))

[('russie', 0.6767497658729553), ('italie', 0.665309727191925), ('afrique', 0.6608356833457947), ('allemagne', 0.6561207175254822), ('grece', 0.6292099952697754), ('egypte', 0.6222110390663147), ('republique', 0.6180391311645508), ('nation', 0.6069801449775696), ('autriche', 0.6021126508712769), ('hollande', 0.598942756652832)]


### Modèle 5: Taille fenêtre 20 (vs 5) et vecteur 32 (vs 100)

In [47]:
%%time
model = Word2Vec(
    corpus, # On passe le corpus de ngrams que nous venons de créer
    vector_size=32, # Le nombre de dimensions dans lesquelles le contexte des mots devra être réduit, aka. vector_size
    window=20, # La taille du "contexte", ici 5 mots avant et après le mot observé
    min_count=5, # On ignore les mots qui n'apparaissent pas au moins 5 fois dans le corpus
    workers=4, # Permet de paralléliser l'entraînement du modèle en 4 threads
    epochs=5 # Nombre d'itérations du réseau de neurones sur le jeu de données pour ajuster les paramètres avec la descende de gradient, aka. epochs.
)

CPU times: user 26min 43s, sys: 1min 14s, total: 27min 58s
Wall time: 9min 51s


In [48]:
outfile = f"../data/bulletins_5.model"
model.save(outfile)

In [49]:
model = Word2Vec.load("../data/bulletins_5.model")

In [50]:
model.wv["bruxelles"]

array([-0.503185  , -0.01135701,  2.9071445 , -0.33935323,  0.7852202 ,
       -7.8717766 , -1.3153803 ,  8.027404  , -1.2895185 , -9.461477  ,
        0.55994606,  7.050791  , -0.3875398 , -0.6959667 ,  3.3068764 ,
       -7.1694727 , -3.735939  ,  6.395983  ,  2.3281946 ,  2.3793006 ,
       -4.2038946 , -7.604685  , -0.70219207,  4.476044  , -0.3055644 ,
       -0.7566809 , -1.6676246 , -1.3390484 , -2.741731  ,  4.5634756 ,
        0.02120536,  0.61037207], dtype=float32)

In [51]:
model.wv.similarity("boucher", "boulanger")

0.8505966

In [52]:
model.wv.most_similar("bruxelles", topn=10)

[('laeken', 0.8055386543273926),
 ('liege', 0.7914955615997314),
 ('gand', 0.7288044095039368),
 ('bruxelloise', 0.7005123496055603),
 ('aux_confins', 0.6939319372177124),
 ('nivelles', 0.6921486854553223),
 ('obbgations_tramways', 0.6838479042053223),
 ('ostende', 0.6827777624130249),
 ('paris', 0.6722395420074463),
 ('louvain', 0.6675084829330444)]

In [53]:
print(model.wv.most_similar(positive=['bruxelles', 'france'], negative=['belgique']))

[('liege', 0.8014519214630127), ('huy', 0.7924149632453918), ('gand', 0.7921144962310791), ('ostende', 0.7839378714561462), ('hambourg', 0.7556935548782349), ('manchester', 0.7534753680229187), ('lille', 0.7480643391609192), ('verviers', 0.7413298487663269), ('jadis', 0.7364925146102905), ('berlin', 0.735415518283844)]


### Modèle 6 : Vecteur 300 et fenêtre 10

In [54]:
%%time
model = Word2Vec(
    corpus, # On passe le corpus de ngrams que nous venons de créer
    vector_size=300, # Le nombre de dimensions dans lesquelles le contexte des mots devra être réduit, aka. vector_size
    window=10, # La taille du "contexte", ici 5 mots avant et après le mot observé
    min_count=5, # On ignore les mots qui n'apparaissent pas au moins 5 fois dans le corpus
    workers=4, # Permet de paralléliser l'entraînement du modèle en 4 threads
    epochs=5 # Nombre d'itérations du réseau de neurones sur le jeu de données pour ajuster les paramètres avec la descende de gradient, aka. epochs.
)

CPU times: user 52min 11s, sys: 2min 14s, total: 54min 25s
Wall time: 16min 7s


In [55]:
outfile = f"../data/bulletins_6.model"
model.save(outfile)

In [56]:
model = Word2Vec.load("../data/bulletins_6.model")

In [57]:
model.wv["bruxelles"]

array([-1.8941535 , -1.8441689 , -1.2983671 , -0.47930893, -1.4705082 ,
        1.243803  ,  0.12675878,  0.46231017, -1.1026258 ,  0.11179759,
        0.067589  ,  0.2038403 , -1.3499236 ,  4.0949454 , -0.36215502,
       -1.1563224 ,  0.82115173,  0.82290274, -0.9631553 , -0.68517464,
       -3.4759614 , -1.2089055 ,  1.694218  ,  1.6561887 , -1.1776484 ,
       -0.30590585, -1.2849115 , -0.8244972 ,  0.4800813 ,  1.1263399 ,
       -0.9571767 ,  1.5167176 ,  2.4747825 , -2.232404  ,  2.8815398 ,
       -3.84935   ,  2.4273005 , -0.0388324 ,  0.07375836, -1.9437244 ,
       -2.5345845 ,  0.32627085, -0.52031565, -1.6569221 , -1.452766  ,
       -4.066375  , -2.7125504 , -2.0605338 ,  2.3531666 , -2.2015738 ,
       -1.3226348 , -0.3980228 ,  0.32416967, -2.3192768 ,  3.910695  ,
       -1.1025765 , -0.64029545, -1.7138597 , -3.528511  ,  1.1143405 ,
       -0.11756355,  0.28521106,  2.1705024 ,  0.03962039,  2.611368  ,
       -0.15945473, -1.0234438 ,  3.7989838 ,  0.32224187,  2.04

In [95]:
model.wv.similarity("voiture", "carrosse")

0.24132355

In [101]:
model.wv.most_similar("boucher", topn=10)

[('tailleur', 0.6623472571372986),
 ('boulanger', 0.6560649275779724),
 ('cabaretier', 0.6487619280815125),
 ('menuisier', 0.6416441798210144),
 ('serrurier', 0.6407414078712463),
 ('charcutier', 0.6258613467216492),
 ('tapissier', 0.614556074142456),
 ('terrassier', 0.6109156012535095),
 ('cordonnier', 0.6100437641143799),
 ('fabricant', 0.6039721965789795)]

In [109]:
print(model.wv.most_similar(positive=['homme', 'roi'], negative=['femme']))

[('peuple', 0.5006588101387024), ('souverain', 0.49426090717315674), ('parlement', 0.489849716424942), ('pouvoir_legislatif', 0.48538702726364136), ('prince', 0.48223361372947693), ('citoyen', 0.4811207354068756), ('magistrat_communal', 0.46513354778289795), ('pouvoir_civil', 0.4647432267665863), ('gouvernement', 0.4601726233959198), ('soldat', 0.4586349129676819)]


In [111]:
print(model.wv.most_similar(positive=['homme', 'bourgmestre'], negative=['femme']))

[('collegue', 0.4746754467487335), ('defenseur', 0.45523303747177124), ('scrupule', 0.44064831733703613), ('conseiller', 0.43790969252586365), ('temoignage', 0.43457233905792236), ('sentiment', 0.4340687692165375), ('orateur', 0.4326702356338501), ('geste', 0.43116092681884766), ('ministre', 0.42899051308631897), ('pouvoir_legislatif', 0.4280470013618469)]


In [112]:
print(model.wv.most_similar(positive=['bourgmestre', 'bruxelles'], negative=['echevin']))

[('berne', 0.39495036005973816), ('nivelles', 0.3824808895587921), ('huy', 0.38201335072517395), ('prague', 0.37479788064956665), ('zurich', 0.3697776198387146), ('liege', 0.3689724802970886), ('turnhout', 0.3591597080230713), ('bruxelle', 0.3570358157157898), ('gand', 0.3554554581642151), ('verviers', 0.3481411039829254)]


### Modèle 7 : 300 dimensions et élargissement de la fenêtre à 40

In [140]:
%%time
model = Word2Vec(
    corpus, # On passe le corpus de ngrams que nous venons de créer
    vector_size=300, # Le nombre de dimensions dans lesquelles le contexte des mots devra être réduit, aka. vector_size
    window=40, # La taille du "contexte", ici 5 mots avant et après le mot observé
    min_count=5, # On ignore les mots qui n'apparaissent pas au moins 5 fois dans le corpus
    workers=4, # Permet de paralléliser l'entraînement du modèle en 4 threads
    epochs=5 # Nombre d'itérations du réseau de neurones sur le jeu de données pour ajuster les paramètres avec la descende de gradient, aka. epochs.
)

CPU times: user 1h 18min 26s, sys: 3min 23s, total: 1h 21min 49s
Wall time: 23min 12s


In [141]:
outfile = f"../data/bulletins_7.model"
model.save(outfile)

In [142]:
model = Word2Vec.load("../data/bulletins_7.model")

In [143]:
model.wv["bruxelles"]

array([ 1.9420298 ,  3.9572117 , -0.479781  ,  1.8512313 ,  0.6077983 ,
        1.3188373 , -1.9745599 ,  0.46632752,  2.8355045 , -0.65798265,
       -3.0141528 ,  0.02807515,  2.7232294 , -3.1294572 , -0.7755505 ,
       -0.8761443 ,  4.6830487 , -0.0317545 , -1.208987  , -4.676162  ,
       -1.0931574 ,  3.129132  , -1.7631922 ,  0.0724907 , -4.6236115 ,
        0.10611718,  1.9277457 ,  3.283433  , -1.8141987 ,  0.96423477,
        0.50486016,  0.76499254,  0.17737277,  1.2694582 ,  1.6600125 ,
       -2.447073  , -4.820738  ,  0.03984297, -2.380742  ,  1.2092748 ,
        3.1448622 , -1.1168028 ,  4.773965  , -3.0806484 , -0.72048235,
       -2.12679   , -4.354698  , -1.9469824 ,  1.847901  , -3.521771  ,
        1.4613243 ,  2.1824555 , -3.1887782 , -0.9928644 ,  2.058186  ,
        0.1577508 , -1.3968624 ,  0.07626353,  0.7373336 , -2.6723197 ,
        2.6736565 , -1.9131659 , -1.7043934 ,  2.637292  ,  2.1176355 ,
       -2.2369292 , -0.32820123, -0.18236516,  1.1420782 , -0.24

In [144]:
model.wv.similarity("boucher", "boulanger")

0.59319586

In [145]:
model.wv.similarity("voiture", "carrosse")

0.1269484

In [146]:
model.wv.most_similar("bruxelles", topn=10)

[('liege', 0.5068586468696594),
 ('nivelles', 0.49116843938827515),
 ('bruxelle', 0.4783177375793457),
 ('xelles', 0.4692355692386627),
 ('tournai', 0.4536599814891815),
 ('lille', 0.44403088092803955),
 ('ostende', 0.44255688786506653),
 ('liruxelles', 0.4339611530303955),
 ('verviers', 0.43019574880599976),
 ('dette_active', 0.4301547706127167)]

In [147]:
model.wv.most_similar("boucher", topn=10)

[('boulanger', 0.5931957364082336),
 ('marchand', 0.579157829284668),
 ('coiffeur', 0.5705497860908508),
 ('etre_abattus', 0.5697841644287109),
 ('fabricant', 0.5636791586875916),
 ('mecanicien', 0.5634440779685974),
 ('menuisier', 0.5607987642288208),
 ('cordonnier', 0.5579670071601868),
 ('serrurier', 0.5513495802879333),
 ('cabaretier', 0.5434521436691284)]

In [148]:
print(model.wv.most_similar(positive=['bruxelles', 'france'], negative=['belgique']))

[('liege', 0.5512763261795044), ('lille', 0.5193424820899963), ('tournai', 0.5141823291778564), ('ostende', 0.5015013217926025), ('huy', 0.48177802562713623), ('prague', 0.4748130738735199), ('gand', 0.47399699687957764), ('lyon', 0.46861395239830017), ('rotterdam', 0.4614550471305847), ('geneve', 0.45686960220336914)]


In [149]:
print(model.wv.most_similar(positive=['homme', 'roi'], negative=['femme']))

[('congres_national', 0.5066206455230713), ('souverain', 0.49232104420661926), ('peuple', 0.4825543463230133), ('citoyen', 0.4798600673675537), ('drapeau_national', 0.4787912964820862), ('prince', 0.47850993275642395), ('homme_politique', 0.4680681824684143), ('roi_albert', 0.46507886052131653), ('premier_magistrat', 0.45432090759277344), ('parlement', 0.4437144100666046)]


In [150]:
print(model.wv.most_similar(positive=['homme', 'bourgmestre'], negative=['femme']))

[('defenseur', 0.43767887353897095), ('collegue', 0.4311828017234802), ('temoignage', 0.42559802532196045), ('conseiller', 0.4247235059738159), ('homme_politique', 0.4245629906654358), ('premier_magistrat', 0.4244137406349182), ('citoyen', 0.4110882580280304), ('seul_instant', 0.4006548821926117), ('instant', 0.3941536247730255), ('journaliste', 0.3907710015773773)]


In [151]:
print(model.wv.most_similar(positive=['bourgmestre', 'bruxelles'], negative=['echevin']))

[('bruxelle', 0.3950777053833008), ('prague', 0.3701161742210388), ('nivelles', 0.366010457277298), ('xelles', 0.35609501600265503), ('binche', 0.3530123829841614), ('tirlemont', 0.35275956988334656), ('duc', 0.35234901309013367), ('zurich', 0.3492000102996826), ('liege', 0.3395634591579437), ('tournai', 0.3369255065917969)]


In [152]:
print(model.wv.most_similar(positive=['bruxelles', 'paris'], negative=['belgique']))

[('liege', 0.4866924583911896), ('londres', 0.4715336859226227), ('gand', 0.4669768512248993), ('amsterdam', 0.4669738709926605), ('rotterdam', 0.4630443751811981), ('berlin', 0.4575584828853607), ('lyon', 0.45622313022613525), ('lille', 0.44650131464004517), ('hambourg', 0.4392347037792206), ('ostende', 0.43219050765037537)]


### Modèle 8 : Vecteur 300, fenêtre 5

In [153]:
%%time
model = Word2Vec(
    corpus, # On passe le corpus de ngrams que nous venons de créer
    vector_size=300, # Le nombre de dimensions dans lesquelles le contexte des mots devra être réduit, aka. vector_size
    window=5, # La taille du "contexte", ici 5 mots avant et après le mot observé
    min_count=5, # On ignore les mots qui n'apparaissent pas au moins 5 fois dans le corpus
    workers=4, # Permet de paralléliser l'entraînement du modèle en 4 threads
    epochs=5 # Nombre d'itérations du réseau de neurones sur le jeu de données pour ajuster les paramètres avec la descende de gradient, aka. epochs.
)

CPU times: user 52min 31s, sys: 2min 45s, total: 55min 16s
Wall time: 17min 52s


In [154]:
outfile = f"../data/bulletins_8.model"
model.save(outfile)

In [155]:
model = Word2Vec.load("../data/bulletins_8.model")

In [156]:
model.wv["bruxelles"]

array([ 0.03376665, -0.15368643, -0.2372689 ,  0.16326897,  0.35034972,
       -0.7852263 , -0.71298736, -1.1754018 , -2.959566  ,  0.9983248 ,
       -0.5194356 ,  0.49538508, -0.64167076,  0.48807636,  2.068091  ,
        0.5359729 ,  1.0986643 ,  0.8097426 , -0.56722987,  1.3344384 ,
       -1.5750489 ,  1.0391806 , -0.02509025,  2.548433  , -1.0727464 ,
       -1.1938587 , -0.31837767, -1.2871847 ,  0.6490244 , -0.00775517,
        0.40511045, -0.4292743 ,  2.499439  , -1.2697452 ,  3.769812  ,
       -0.14065114,  1.5113703 ,  1.7290576 ,  1.3724093 ,  1.6587178 ,
       -0.47041377,  1.3847239 ,  0.7703857 , -0.55137706, -1.044063  ,
       -2.366426  , -2.2340603 , -0.47734085,  0.6138347 , -1.1245146 ,
       -1.4871107 ,  2.4642067 ,  2.8817992 , -1.2213827 , -0.79821867,
       -1.72223   ,  0.71573466, -1.1036998 , -1.679436  ,  0.19673033,
        0.86890656, -1.2333289 , -0.06400593, -0.61558604,  1.071806  ,
       -0.3724426 , -0.16024712,  1.7617979 ,  0.700619  ,  1.86

In [157]:
model.wv.similarity("boucher", "boulanger")

0.6921761

In [158]:
model.wv.similarity("voiture", "carrosse")

0.20062777

In [159]:
model.wv.most_similar("bruxelles", topn=10)

[('nivelles', 0.5794664025306702),
 ('rruxelles', 0.5737338662147522),
 ('liege', 0.5225680470466614),
 ('gand', 0.5023678541183472),
 ('paris', 0.48170360922813416),
 ('xelles', 0.4629574120044708),
 ('malines', 0.4540492296218872),
 ('londres', 0.450400710105896),
 ('lille', 0.449739545583725),
 ('verviers', 0.44773247838020325)]

In [160]:
model.wv.most_similar("boucher", topn=10)

[('menuisier', 0.705475926399231),
 ('tailleur', 0.7049712538719177),
 ('cordonnier', 0.6925906538963318),
 ('boulanger', 0.6921761631965637),
 ('charcutier', 0.6875828504562378),
 ('tapissier', 0.6866316795349121),
 ('serrurier', 0.67592453956604),
 ('marbrier', 0.657673180103302),
 ('cabaretier', 0.6564407348632812),
 ('batelier', 0.6554194092750549)]

In [161]:
print(model.wv.most_similar(positive=['bruxelles', 'france'], negative=['belgique']))

[('verviers', 0.534920334815979), ('londres', 0.5326730608940125), ('liege', 0.5236067175865173), ('tournai', 0.5141814947128296), ('gand', 0.5092669129371643), ('hambourg', 0.4946388006210327), ('nivelles', 0.4940313696861267), ('paris', 0.4934212565422058), ('prague', 0.4888829290866852), ('lille', 0.4820585250854492)]


In [162]:
print(model.wv.most_similar(positive=['homme', 'roi'], negative=['femme']))

[('substitut', 0.4808488190174103), ('prince', 0.4788137674331665), ('peuple', 0.4772094190120697), ('pouvoir_legislatif', 0.47451895475387573), ('parlement', 0.47124239802360535), ('congres_national', 0.46308228373527527), ('magistrat', 0.45981365442276), ('portique', 0.45897307991981506), ('parti_liberal', 0.4527781903743744), ('souverain', 0.4506208300590515)]


In [163]:
print(model.wv.most_similar(positive=['homme', 'bourgmestre'], negative=['femme']))

[('collegue', 0.4900236427783966), ('defenseur', 0.481063574552536), ('scrupule', 0.4779437482357025), ('geste', 0.4597097337245941), ('discours', 0.4495963454246521), ('collaborateur', 0.447189062833786), ('orateur', 0.4450376033782959), ('juriste', 0.4424530267715454), ('ministre', 0.44214901328086853), ('sentiment', 0.44033125042915344)]


In [164]:
print(model.wv.most_similar(positive=['bourgmestre', 'bruxelles'], negative=['echevin']))

[('nivelles', 0.434680312871933), ('rruxelles', 0.41497689485549927), ('liege', 0.39032211899757385), ('paris', 0.3756389915943146), ('prague', 0.3523339331150055), ('termonde', 0.3519954979419708), ('xelles', 0.35197094082832336), ('bruges', 0.34980642795562744), ('izier', 0.3487500846385956), ('zurich', 0.3485882878303528)]


In [165]:
print(model.wv.most_similar(positive=['bruxelles', 'paris'], negative=['belgique']))

[('gand', 0.5362998843193054), ('londres', 0.5179358720779419), ('liege', 0.4998849332332611), ('verviers', 0.48312634229660034), ('nivelles', 0.46410995721817017), ('rruxelles', 0.4593343436717987), ('tournai', 0.4545356333255768), ('prague', 0.45003101229667664), ('amsterdam', 0.44772326946258545), ('ostende', 0.4376828074455261)]


### Modèle 9 : Vecteur 100, fenêtre 40

In [166]:
%%time
model = Word2Vec(
    corpus, # On passe le corpus de ngrams que nous venons de créer
    vector_size=100, # Le nombre de dimensions dans lesquelles le contexte des mots devra être réduit, aka. vector_size
    window=40, # La taille du "contexte", ici 5 mots avant et après le mot observé
    min_count=5, # On ignore les mots qui n'apparaissent pas au moins 5 fois dans le corpus
    workers=4, # Permet de paralléliser l'entraînement du modèle en 4 threads
    epochs=5 # Nombre d'itérations du réseau de neurones sur le jeu de données pour ajuster les paramètres avec la descende de gradient, aka. epochs.
)

CPU times: user 42min 18s, sys: 2min 4s, total: 44min 23s
Wall time: 14min 49s


In [167]:
outfile = f"../data/bulletins_9.model"
model.save(outfile)

In [168]:
model = Word2Vec.load("../data/bulletins_9.model")

In [169]:
model.wv["bruxelles"]

array([-5.632714  , -2.0852008 , -2.3220026 ,  2.9778798 , -3.181986  ,
        1.9977478 , -2.6852753 , -3.832645  ,  0.04784409,  3.4816964 ,
        0.8636748 ,  1.9691361 ,  0.9180136 ,  3.7392008 ,  1.2560499 ,
        1.4207648 ,  4.804548  , -1.0894951 ,  9.480927  , -0.43994424,
        5.1391587 , -3.0354512 , -5.303183  , -0.57952183,  0.13835198,
        2.1639357 ,  3.3303363 ,  1.3061031 ,  2.9484484 , -4.6935205 ,
       -4.3497524 ,  3.09033   , -4.0258107 , -0.93852985,  2.2096207 ,
        3.5929976 , -1.8873901 ,  5.338172  , -0.7563687 , -2.4372873 ,
        1.5720205 , -1.5531121 , -0.79097915,  6.760926  ,  0.7869648 ,
       -3.91618   ,  2.8180945 , -8.252442  ,  3.1905584 , -1.4279473 ,
        0.6705358 ,  2.1984186 ,  1.9810268 , -2.109154  , -3.2923827 ,
        2.5102317 , -3.6633947 , -7.3272233 ,  1.1219487 ,  3.597578  ,
        0.76092094,  2.7240448 ,  5.8935547 ,  4.804508  ,  2.0144448 ,
        4.04141   ,  1.5511286 ,  4.4134827 ,  1.5147197 ,  0.48

In [170]:
model.wv.similarity("boucher", "boulanger")

0.68535304

In [171]:
model.wv.similarity("voiture", "carrosse")

0.12812918

In [172]:
model.wv.most_similar("bruxelles", topn=10)

[('xelles', 0.6651670336723328),
 ('liege', 0.5861473083496094),
 ('nivelles', 0.5695111155509949),
 ('laeken', 0.563617467880249),
 ('dette_active', 0.5545754432678223),
 ('bruxelle', 0.5514754056930542),
 ('bruxe', 0.540813148021698),
 ('manufacturiere', 0.5400780439376831),
 ('huy', 0.5379692316055298),
 ('ostende', 0.5323569774627686)]

In [173]:
model.wv.most_similar("boucher", topn=10)

[('cordonnier', 0.7136046290397644),
 ('fabricant', 0.695364773273468),
 ('boulanger', 0.6853529810905457),
 ('charpentier', 0.6758134365081787),
 ('marchand', 0.6743961572647095),
 ('serrurier', 0.6743265390396118),
 ('menuisier', 0.6717424988746643),
 ('charcutier', 0.6633214950561523),
 ('tailleur', 0.656404435634613),
 ('meunier', 0.6544731855392456)]

In [174]:
print(model.wv.most_similar(positive=['bruxelles', 'france'], negative=['belgique']))

[('huy', 0.6195623278617859), ('hambourg', 0.5991153717041016), ('manchester', 0.5982712507247925), ('liege', 0.5854235887527466), ('londres', 0.5793405175209045), ('ostende', 0.5688986778259277), ('paris', 0.5649790167808533), ('lille', 0.5636627674102783), ('amsterdam', 0.5562283992767334), ('gand', 0.5400305390357971)]


In [175]:
print(model.wv.most_similar(positive=['homme', 'roi'], negative=['femme']))

[('congres_national', 0.6028381586074829), ('prince', 0.6000238060951233), ('congres', 0.5982035398483276), ('souverain', 0.5863264799118042), ('peuple', 0.5856724977493286), ('magistrat', 0.5816968083381653), ('citoyen', 0.5683267116546631), ('drapeau_national', 0.5603322982788086), ('banquet', 0.5595159530639648), ('congo', 0.5558876395225525)]


In [176]:
print(model.wv.most_similar(positive=['homme', 'bourgmestre'], negative=['femme']))

[('nistre', 0.5475883483886719), ('defenseur', 0.5099793672561646), ('collegue', 0.49658870697021484), ('coliege', 0.4927530288696289), ('premier_ministre', 0.48967066407203674), ('rapporteur', 0.48401108384132385), ('g_mestre', 0.48272135853767395), ('conseiller', 0.4821358323097229), ('discours', 0.47750771045684814), ('premier_magistrat', 0.4689492881298065)]


In [177]:
print(model.wv.most_similar(positive=['bourgmestre', 'bruxelles'], negative=['echevin']))

[('bruxe', 0.4750226140022278), ('bruxelle', 0.47379934787750244), ('laeken', 0.4509906768798828), ('manufacturiere', 0.44638532400131226), ('xelles', 0.44368958473205566), ('traditionnel_defile', 0.4164222180843353), ('tournai', 0.41501978039741516), ('prague', 0.412350058555603), ('huy', 0.4119340777397156), ('otel', 0.4079040586948395)]


In [178]:
print(model.wv.most_similar(positive=['bruxelles', 'paris'], negative=['belgique']))

[('hambourg', 0.5637348890304565), ('londres', 0.5500701069831848), ('manchester', 0.532948911190033), ('gand', 0.5250847935676575), ('liege', 0.5164158940315247), ('berlin', 0.5028001666069031), ('huy', 0.491977334022522), ('ostende', 0.4851594865322113), ('amsterdam', 0.48425236344337463), ('laeken', 0.4806765615940094)]
